<a href="https://colab.research.google.com/github/Albinator3000/OIDD255_RealEstate_RAG_Model/blob/main/AI_Property_Due_Diligence_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Property Due Diligence Assistant
### OIDD 2550 - Lab 5: LLM Pitch Project

**RAG-based system for real estate investment analysis**

---

## Quick Start Guide

1. **Set Runtime**: Runtime → Change runtime type → **T4 GPU** (free tier)
2. **Choose Your LLM**: Uncomment ONE of the three options below:
   - **Option 1**: Free HuggingFace (Phi-2) - No API key needed ⭐ **EASIEST**
   - **Option 2**: OpenAI (Paid) - Best quality
   - **Option 3**: Groq (Free) - Fast & high quality ⭐ **RECOMMENDED**
3. **Run All**: Runtime → Run all (Ctrl+F9)

---

## Step 1: Install Dependencies

This installs all required packages with OpenTelemetry pinned to version 1.37.0 for compatibility.

In [26]:
# Install all dependencies (this takes ~2 minutes)
!pip install \
    transformers accelerate \
    langchain langchain-huggingface langchain-community langchain-text-splitters \
    chromadb \
    pandas numpy matplotlib seaborn plotly \
    PyPDF2 python-docx \
    sentence-transformers scikit-learn \
    opentelemetry-api==1.37.0 \
    opentelemetry-sdk==1.37.0 \
    opentelemetry-exporter-otlp-proto-common==1.37.0 \
    opentelemetry-exporter-otlp-proto-grpc==1.37.0 \
    opentelemetry-proto==1.37.0 \
    opentelemetry-semantic-conventions==0.58b0

print("✅ All dependencies installed")

✅ All dependencies installed


## Step 2: Import Libraries

In [27]:
import os
import warnings
warnings.filterwarnings('ignore')

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Step 3: Choose Your LLM

### ⚠️ IMPORTANT: Uncomment ONLY ONE option below!

### Option 1: HuggingFace Phi-2 (Free, No API Key) ⭐ EASIEST

**Pros:** Completely free, no setup  
**Cons:** Lower quality than Llama 3.1, slower inference

**Instructions:** Just run the cell below!

In [29]:
# OPTION 1: HuggingFace Phi-2 (uncomment to use)

# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain_huggingface import HuggingFacePipeline

# print("🔄 Loading Phi-2 model (2-3 minutes)...")

# model_name = "microsoft/phi-2"
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     device_map="auto",
#     load_in_8bit=True
# )

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     temperature=0.3,
#     top_p=0.95,
#     repetition_penalty=1.15
# )

# llm = HuggingFacePipeline(pipeline=pipe)
# print("✅ Phi-2 model loaded and ready!")

### Option 2: OpenAI API (Paid, Best Quality)

**Pros:** GPT-3.5/4, excellent results  
**Cons:** Costs money (~$0.10 per analysis)

**Instructions:**
1. Get API key: https://platform.openai.com
2. In Colab: Click 🔑 icon (Secrets) → Add new secret
3. Name: `OPENAI_API_KEY`, Value: your API key
4. Toggle "Notebook access" ON

In [ ]:
# OPTION 2: OpenAI (uncomment to use)

# !pip install -q openai langchain-openai

# from langchain_openai import ChatOpenAI
# from google.colab import userdata

# api_key = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     temperature=0.3,
#     api_key=api_key
# )

# print("✅ OpenAI GPT-3.5 ready!")

### Option 3: Groq API (Free, Fast) ⭐ RECOMMENDED

**Pros:** Uses Llama 3.1 70B, very fast, free tier  
**Cons:** Requires API key (takes 30 seconds to get)

**Instructions:**
1. Get free API key: https://console.groq.com (sign up with Google/GitHub)
2. In Colab: Click 🔑 icon (Secrets) → Add new secret
3. Name: `GROQ_API_KEY`, Value: your API key (starts with `gsk_...`)
4. Toggle "Notebook access" ON

In [30]:
# OPTION 3: Groq (uncomment to use) ⭐ RECOMMENDED

!pip install -q groq langchain-groq

from langchain_groq import ChatGroq
from google.colab import userdata

api_key = userdata.get('GROQ_API_KEY_OIDD255')
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    api_key=api_key
)

print("✅ Groq (Llama 3.1 8B) ready!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 8.7 MB/s eta 0:00:00
✅ Groq (Llama 3.1 8B) ready!


---

## Step 4: Configure System

In [31]:
# Configuration
CONFIG = {
    'embedding_model': 'sentence-transformers/all-MiniLM-L6-v2',
    'chunk_size': 1000,
    'chunk_overlap': 200,
    'vector_db_path': './chroma_db',
    'top_k': 5
}

# Risk category weights
RISK_WEIGHTS = {
    'Structural': 0.30,
    'Financial': 0.30,
    'Legal': 0.20,
    'Operational': 0.10,
    'Market': 0.10
}

print("✅ Configuration set")

✅ Configuration set


## Step 5: Initialize Embeddings & Text Splitter

In [32]:
print("🔄 Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name=CONFIG['embedding_model']
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CONFIG['chunk_size'],
    chunk_overlap=CONFIG['chunk_overlap']
)

print("✅ Embeddings ready")

🔄 Loading embedding model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embeddings ready


## Step 6: Load Sample Property Data

This demo uses a synthetic 4-unit multifamily property in Austin, TX

In [33]:
# Sample Property Details
SAMPLE_PROPERTY = {
    'address': '1234 Oak Street, Austin, TX 78701',
    'property_type': 'Multifamily - 4 units',
    'asking_price': 425000,
    'year_built': 1985,
    'sqft': 3200,
}

# Sample Inspection Report
SAMPLE_INSPECTION = """
PROPERTY INSPECTION REPORT
Date: November 15, 2024
Property: 1234 Oak Street, Austin, TX 78701

CRITICAL ISSUES:
1. HVAC System - Unit 2: 18 years old, refrigerant leak detected.
   Estimated remaining life: 1-2 years. Replacement cost: $5,000-$7,000.

2. Roof: Asphalt shingle roof installed in 2005 (19 years old).
   Multiple missing/damaged shingles. Minor water staining in attic.
   Estimated remaining life: 2-3 years. Replacement cost: $12,000-$15,000.

MODERATE ISSUES:
3. Electrical Panel: 60 amp service (outdated). Upgrade to 200 amp recommended.
   Cost: $3,000-$4,000.

4. Water Heaters - Units 1 & 3: Both over 10 years old.
   Replacement within 2-3 years recommended. Cost: $1,200 each.

5. Foundation: Minor hairline cracks in southeast corner.
   No active movement. Monitor for changes. Potential repair cost: $2,000-$5,000.

TOTAL ESTIMATED DEFERRED MAINTENANCE: $25,000 - $35,000 over next 2-3 years
IMMEDIATE REPAIRS RECOMMENDED: $8,000 - $12,000 (HVAC Unit 2, roof patch)
"""

# Real Estate Domain Knowledge
DOMAIN_KNOWLEDGE = """
REAL ESTATE VALUATION PRINCIPLES:

Cap Rate Analysis:
- Cap Rate = Net Operating Income (NOI) / Property Value
- Austin multifamily market: typically 4.5% - 6.5%
- Lower cap rates = higher property values, lower returns
- Higher cap rates = lower property values, potentially higher risk

Common Red Flags:
1. Structural Issues:
   - Foundation cracks > 1/4 inch
   - Active mold growth
   - Roof leaks or major damage
   - HVAC systems older than 15 years

2. Financial Red Flags:
   - Vacancy rates above 15%
   - Delinquencies over 30 days
   - Operating expense ratios above 50%
   - Underreported maintenance costs

3. Legal Red Flags:
   - Unpermitted additions/renovations
   - Zoning violations
   - Non-standard lease terms

4. Operational Red Flags:
   - High tenant turnover
   - Month-to-month leases
   - Poor property management

Renovation Cost Estimates (Austin market, 2024):
- HVAC replacement: $5,000 - $8,000 per unit
- Roof replacement: $8,000 - $15,000
- Water heater: $1,000 - $1,500 per unit
- Foundation repair: $2,000 - $10,000
- Electrical panel upgrade: $2,500 - $5,000
"""

# Sample Financials
SAMPLE_FINANCIALS = """
OPERATING STATEMENT - 2024
Property: 1234 Oak Street, Austin, TX 78701

INCOME:
Gross Scheduled Income: $55,200 (4 units × $1,150 avg × 12 months)
Vacancy Loss (10%): -$5,520
Gross Operating Income: $49,680

OPERATING EXPENSES:
Property Taxes: $6,400
Insurance: $2,400
Utilities (Landlord-paid): $1,800
Repairs & Maintenance: $3,000
Property Management (10%): $4,968
Landscaping: $1,200
Pest Control: $600
Other: $800

Total Operating Expenses: $21,168
Net Operating Income (NOI): $28,512

Asking Price: $425,000
Implied Cap Rate: 6.71%
"""

print("✅ Sample data loaded")
print(f"\n📋 Property: {SAMPLE_PROPERTY['address']}")
print(f"💰 Asking Price: ${SAMPLE_PROPERTY['asking_price']:,}")

✅ Sample data loaded

📋 Property: 1234 Oak Street, Austin, TX 78701
💰 Asking Price: $425,000


## Step 7: Create Vector Database (RAG System)

In [34]:
# Create documents from sample data
documents = [
    Document(
        page_content=SAMPLE_INSPECTION,
        metadata={'source': 'inspection_report', 'type': 'inspection'}
    ),
    Document(
        page_content=DOMAIN_KNOWLEDGE,
        metadata={'source': 'knowledge_base', 'type': 'domain'}
    ),
    Document(
        page_content=SAMPLE_FINANCIALS,
        metadata={'source': 'financial_statement', 'type': 'financials'}
    )
]

# Split documents into chunks
print("🔄 Creating vector database...")
split_docs = text_splitter.split_documents(documents)

# Create vector store
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=CONFIG['vector_db_path']
)

print(f"✅ Vector database created with {len(split_docs)} chunks")

🔄 Creating vector database...
✅ Vector database created with 4 chunks


## Step 8: Create RAG Chain

In [36]:
# Create custom prompt for real estate analysis
PROMPT_TEMPLATE = """
You are an expert real estate analyst. Use the context below to answer the question.
Be specific and cite numbers from the documents.

Context:
{context}

Question: {input}

Answer:
"""

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "input"]
)

# Create the document chain
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Create RAG chain
qa_chain = create_retrieval_chain(
    vectorstore.as_retriever(search_kwargs={"k": CONFIG['top_k']}),
    combine_docs_chain
)

print("✅ RAG chain ready!")

NameError: name 'create_stuff_documents_chain' is not defined

## Step 9: Test the System

In [ ]:
test_query = "What are the critical issues found in the inspection and their estimated costs?"

print(f"🔍 Query: {test_query}\n")
print("🤖 Generating response...\n")

result = qa_chain.invoke({"input": test_query})

print("="*80)
print("ANSWER:")
print("="*80)
print(result['result'])
print("\n" + "="*80)

## Step 10: Risk Assessment

In [ ]:
def calculate_risk_scores():
    """
    Calculate risk scores based on property analysis.
    In production, these would be generated by LLM analysis.
    """
    scores = {
        'Structural': 35,      # Major issues (roof, HVAC)
        'Financial': 55,       # High vacancy, delinquency
        'Legal': 90,           # Clean leases, no violations
        'Operational': 75,     # Manageable tenant issues
        'Market': 95           # Austin market is strong
    }

    # Calculate weighted overall score
    overall = sum(scores[cat] * RISK_WEIGHTS[cat] for cat in scores.keys())

    return scores, round(overall, 1)

scores, overall = calculate_risk_scores()

print("📊 RISK ASSESSMENT REPORT")
print("="*80)
for category, score in scores.items():
    weight = RISK_WEIGHTS[category]
    status = "✅" if score >= 80 else "⚠️" if score >= 60 else "🔴"
    print(f"{status} {category:15} {score:3}/100  (weight: {weight:.0%})")
print("="*80)
print(f"OVERALL SCORE: {overall}/100")
print("="*80)

if overall >= 75:
    decision = "✅ STRONG GO"
elif overall >= 60:
    decision = "⚠️  PROCEED WITH CAUTION"
else:
    decision = "🔴 NO GO"

print(f"\nRECOMMENDATION: {decision}")

## Step 11: Visualize Risk Scores

In [ ]:
# Risk score visualization
fig, ax = plt.subplots(figsize=(12, 6))

categories = list(scores.keys())
values = list(scores.values())
colors = ['#ff4444' if v < 60 else '#ffaa00' if v < 80 else '#44ff44' for v in values]

ax.barh(categories, values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax.set_xlabel('Risk Score (0-100)', fontsize=12, fontweight='bold')
ax.set_title('Risk Assessment by Category', fontsize=14, fontweight='bold', pad=20)
ax.set_xlim(0, 100)
ax.axvline(x=60, color='orange', linestyle='--', alpha=0.5, linewidth=2, label='Moderate Threshold')
ax.axvline(x=80, color='green', linestyle='--', alpha=0.5, linewidth=2, label='Low Risk Threshold')
ax.legend(loc='lower right')
ax.grid(axis='x', alpha=0.3)

# Add score labels
for i, (cat, score) in enumerate(zip(categories, values)):
    ax.text(score + 2, i, f'{score:.0f}', va='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

print("✅ Risk visualization complete!")

## Step 12: Valuation Analysis

In [ ]:
# Valuation calculations
asking_price = 425000
reported_noi = 28512
deferred_maintenance = 30000  # midpoint of $25k-$35k range
immediate_repairs = 10000     # midpoint of $8k-$12k range

# Adjust NOI for realistic vacancy (10% instead of current 25%)
adjusted_noi = 26000  # More realistic after fixing vacancy

# Calculate fair value at market cap rate (6.5%)
target_cap_rate = 0.065
fair_value = adjusted_noi / target_cap_rate

# Recommended offer (fair value minus immediate repairs)
recommended_offer = fair_value - immediate_repairs

print("💰 VALUATION ANALYSIS")
print("="*80)
print(f"Asking Price:              ${asking_price:,}")
print(f"Reported NOI:              ${reported_noi:,}")
print(f"Adjusted NOI (realistic):  ${adjusted_noi:,}")
print(f"\nFair Market Value (6.5% cap): ${fair_value:,.0f}")
print(f"Less: Immediate Repairs:      -${immediate_repairs:,}")
print(f"\nRecommended Offer:         ${recommended_offer:,.0f}")
print(f"\nDiscount from Ask:         ${asking_price - recommended_offer:,.0f} ({(asking_price - recommended_offer)/asking_price:.1%})")
print("="*80)

print("\n📋 NEGOTIATION STRATEGY:")
print("1. Opening offer: $390,000 (leave room to negotiate up)")
print("2. Request $10,000 seller credit for HVAC replacement")
print("3. Budget $25k-35k for deferred maintenance over next 2-3 years")
print("4. Walk-away price: $410,000")
print("5. Key leverage points:")
print("   - 19-year-old roof needs replacement")
print("   - Failed HVAC in Unit 2")
print("   - Current 25% vacancy rate")
print("   - Delinquent tenant in Unit B")

## Step 13: Overall Score Gauge

In [ ]:
# Create gauge visualization
fig = go.Figure(go.Indicator(
    mode="gauge+number+delta",
    value=overall,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Overall Property Risk Score", 'font': {'size': 24}},
    delta={'reference': 75, 'increasing': {'color': "green"}},
    gauge={
        'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "darkblue"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 40], 'color': '#ffcccc'},
            {'range': [40, 60], 'color': '#ffe6cc'},
            {'range': [60, 80], 'color': '#fff4cc'},
            {'range': [80, 100], 'color': '#ccffcc'}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': 60
        }
    }
))

fig.update_layout(height=400, font={'size': 16})
fig.show()

## Summary

### What This System Does:

✅ **Document Processing**: Analyzes inspection reports, financials, and domain knowledge  
✅ **RAG Architecture**: Uses vector database + LLM for intelligent retrieval  
✅ **Risk Assessment**: Multi-category scoring (Structural, Financial, Legal, Operational, Market)  
✅ **Valuation**: Adjusts NOI, calculates fair value, recommends offer price  
✅ **Go/No-Go Decision**: Data-driven investment recommendation  

### Technical Stack:

- **LLM Options**: Phi-2 (free) / OpenAI (paid) / Groq (free + fast)
- **Embeddings**: sentence-transformers/all-MiniLM-L6-v2
- **Vector DB**: ChromaDB
- **Framework**: LangChain
- **OpenTelemetry**: Version 1.37.0 (pinned for compatibility)

### Business Impact:

- **95% faster** than traditional due diligence (2 minutes vs 2-6 weeks)
- **90% cheaper** ($29-49 vs $2,000-10,000)
- **Scalable**: Analyze 100 properties in the time it takes to analyze 1 manually

---

**Ready for your OIDD 2550 presentation! 🚀**

# Task
Correct the import statement for `RetrievalQA` in cell `DuYq0_naN-1b`.

## fix_retrieval_qa_import

### Subtask:
Correct the import statement for `RetrievalQA` in cell `DuYq0_naN-1b` from `from langchain_community.chains import RetrievalQA` to `from langchain.chains.retrieval_qa import RetrievalQA`.


## Summary:

### Insights or Next Steps
* The import statement for `RetrievalQA` was successfully corrected from `from langchain_community.chains import RetrievalQA` to `from langchain.chains.retrieval_qa import RetrievalQA`.
